In [24]:
# Import Packages

import plotly.express as px
import panel as pn
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv

In [25]:
# Set up Panel Plotly extension

pn.extension('plotly')

In [26]:
# Import hvplot.pandas after pn.extension
# This avoids plotly initialization failure

import hvplot.pandas

In [44]:
# Read the Mapbox API key

load_dotenv()
map_box_api = os.getenv("mapbox")

# Set token using Plotly Express set function

px.set_mapbox_access_token(map_box_api)

# Read in data

florida_median_home_value = pd.read_csv(Path("Florida-Median-Home-Prices-By-County-Q1-2021.csv")).drop_duplicates()

# Change values from string to float, remove $

florida_median_home_value[florida_median_home_value.columns[1:]] = florida_median_home_value[florida_median_home_value.columns[1:]].replace('[\$,]', '', regex=True).astype(float)

lat_long = pd.read_csv(Path("lat_long.csv")).drop_duplicates()
landfall_probability = pd.read_csv(Path("landfall_prob_by_county_50yr.csv")).drop_duplicates()

florida_median_home_value_with_index = florida_median_home_value.set_index("County")
lat_long_with_index = lat_long.set_index("County")
landfall_probability_with_index = landfall_probability.set_index("County")

home_values_with_landfall_probability = (
    pd.concat([florida_median_home_value_with_index, lat_long_with_index, landfall_probability_with_index], axis="columns", sort=True)#1
    .dropna()
    .reset_index()
)

In [56]:
lat_long_with_index.reset_index(inplace=True)
lat_long_with_index["County"][0]

'Alachua'

In [28]:
# Create plots

median_home_value_plot = px.scatter_mapbox(
    home_values_with_landfall_probability,
    lat="Lat",
    lon="Long",
    size="Median Home Price Q1 2021",
    color="County",
    color_continuous_scale=px.colors.cyclical.IceFire,
    title="Florida Median Home Value",
    zoom=3,
    width=1000,
)

tropical_storm_probability_plot = px.scatter_mapbox(
    home_values_with_landfall_probability,
    lat="Lat",
    lon="Long",
    size="Probability of Tropical Storm-Force (>= 40 mph) Wind Gusts in the County",
    color="County",
    color_continuous_scale=px.colors.cyclical.IceFire,
    title="Tropical Storm Landfall Probability",
    zoom=3,
    width=1000,
)

hurricane_probability_plot = px.scatter_mapbox(
    home_values_with_landfall_probability,
    lat="Lat",
    lon="Long",
    size="Probability of Hurricane-Force (>= 75 mph) Wind Gusts in the County",
    color="County",
    color_continuous_scale=px.colors.cyclical.IceFire,
    title="Hurricane Landfall Probability",
    zoom=3,
    width=1000,
)

intense_hurricane_probability_plot = px.scatter_mapbox(
    home_values_with_landfall_probability,
    lat="Lat",
    lon="Long",
    size="Probability of Intense Hurricane-Force (>= 115 mph) Wind Gusts in the County",
    color="County",
    color_continuous_scale=px.colors.cyclical.IceFire,
    title="Intense Hurricane Landfall Probability",
    zoom=3,
    width=1000,
)

median_home_value_correlated_with_tropical_storm_landfall = home_values_with_landfall_probability.hvplot.scatter(
    x="Probability of Tropical Storm-Force (>= 40 mph) Wind Gusts in the County",
    y="Median Home Price Q1 2021",
    title="Florida Median Home Value and Tropical Storm Landfall Correlation",
    width=1000,
).opts(yformatter="%.0f")

median_home_value_correlated_with_hurricane_landfall = home_values_with_landfall_probability.hvplot.scatter(
    x="Probability of Hurricane-Force (>= 75 mph) Wind Gusts in the County",
    y="Median Home Price Q1 2021",
    title="Florida Median Home Value and Hurricane Landfall Correlation",
    width=1000,
).opts(yformatter="%.0f")

median_home_value_correlated_with_intense_hurricane_landfall = home_values_with_landfall_probability.hvplot.scatter(
    x="Probability of Intense Hurricane-Force (>= 115 mph) Wind Gusts in the County",
    y="Median Home Price Q1 2021",
    title="Florida Median Home Value and Intense Hurricane Landfall Correlation",
    width=1000,
).opts(yformatter="%.0f")

In [29]:
# Dashboard Layout

geo_column = pn.Column("## Median Home Value and Hurricane Landfall Probability Geo Plots", median_home_value_plot, hurricane_probability_plot)

scatter_column = pn.Column(
    "## Correlation of Median Home Value and Intense Hurricane Landfall Probability Plots", median_home_value_plot, intense_hurricane_probability_plot)

In [30]:
# Show Panel

storm_home_value_dashboard = pn.Tabs(
    (
        "Geospatial",
        geo_column
    ),
    (
        "Correlations",
        scatter_column
    )
)

storm_home_value_dashboard

ValueError: Out of range float values are not JSON compliant

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)